In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]


In [6]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [8]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy(),'mse'],
)

In [11]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    verbose=1,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/2
782/782 [==============================] - 1s 2ms/step - loss: 0.0740 - sparse_categorical_accuracy: 0.9767 - mse: 27.3582 - val_loss: 0.0991 - val_sparse_categorical_accuracy: 0.9712 - val_mse: 27.5540
Epoch 2/2
782/782 [==============================] - 1s 2ms/step - loss: 0.0629 - sparse_categorical_accuracy: 0.9809 - mse: 27.3587 - val_loss: 0.1032 - val_sparse_categorical_accuracy: 0.9708 - val_mse: 27.5549


In [12]:
history.history

{'loss': [0.07402514666318893, 0.06285056471824646],
 'sparse_categorical_accuracy': [0.9767400026321411, 0.9808800220489502],
 'mse': [27.358200073242188, 27.35873031616211],
 'val_loss': [0.09911438822746277, 0.10318354517221451],
 'val_sparse_categorical_accuracy': [0.9711999893188477, 0.97079998254776],
 'val_mse': [27.553958892822266, 27.554887771606445]}

In [14]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc, mse:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


Evaluate on test data
79/79 [==============================] - 0s 1ms/step - loss: 0.1078 - sparse_categorical_accuracy: 0.9691 - mse: 27.3373
test loss, test acc, mse: [0.10782857984304428, 0.9690999984741211, 27.337263107299805]
Generate predictions for 3 samples
predictions shape: (3, 10)


In [15]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model


In [17]:
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred))

In [20]:
model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
#print(y_train_one_hot[:10])
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
782/782 [==============================] - 1s 1ms/step - loss: 0.0156


# Custom Loss

In [21]:
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)


782/782 [==============================] - 1s 1ms/step - loss: 0.0389


# Custom Metrics

In [22]:
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.0)


model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=3)


Epoch 1/3
782/782 [==============================] - 1s 2ms/step - loss: 0.3464 - categorical_true_positives: 45134.0000
Epoch 2/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1653 - categorical_true_positives: 47564.0000
Epoch 3/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1180 - categorical_true_positives: 48234.0000


# Custom ActivityRegularization 

In [23]:
class ActivityRegularizationLayer(layers.Layer):
    def call(self, inputs):
        self.add_loss(tf.reduce_sum(inputs) * 0.1)
        return inputs  # Pass-through layer.


inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

# The displayed loss will be much higher than before
# due to the regularization component.
model.fit(x_train, y_train, batch_size=64, epochs=1)


782/782 [==============================] - 1s 2ms/step - loss: 2.4784


In [25]:
class MetricLoggingLayer(layers.Layer):
    def call(self, inputs):
        # The `aggregation` argument defines
        # how to aggregate the per-batch values
        # over each epoch:
        # in this case we simply average them.
        self.add_metric(
            keras.backend.std(inputs), name="std_of_activation", aggregation="mean"
        )
        return inputs  # Pass-through layer.


inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Insert std logging as a layer.
x = MetricLoggingLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)
model.summary()

782/782 [==============================] - 1s 2ms/step - loss: 0.3452 - std_of_activation: 0.9782
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
metric_logging_layer_1 (Metr (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x2 = layers.Dense(64, activation="relu", name="dense_2")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

model.add_loss(tf.reduce_sum(x1) * 0.1)

model.add_metric(keras.backend.std(x1), name="std_of_activation", aggregation="mean")

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)


782/782 [==============================] - 1s 2ms/step - loss: 2.4721 - std_of_activation: 0.0018


# Tensor Dataset

In [27]:
model = get_compiled_model()

# First, let's create a training Dataset instance.
# For the sake of our example, we'll use the same MNIST data as before.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))


Epoch 1/3
782/782 [==============================] - 1s 2ms/step - loss: 0.3433 - sparse_categorical_accuracy: 0.9030
Epoch 2/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1674 - sparse_categorical_accuracy: 0.9494
Epoch 3/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1229 - sparse_categorical_accuracy: 0.9634
Evaluate
157/157 [==============================] - 0s 867us/step - loss: 0.1208 - sparse_categorical_accuracy: 0.9636


{'loss': 0.12083735316991806,
 'sparse_categorical_accuracy': 0.9635999798774719}

In [28]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Only use the 100 batches per epoch (that's 64 * 100 samples)
model.fit(train_dataset, epochs=3, steps_per_epoch=100)


Epoch 1/3
100/100 [==============================] - 0s 2ms/step - loss: 0.8092 - sparse_categorical_accuracy: 0.7795
Epoch 2/3
100/100 [==============================] - 0s 2ms/step - loss: 0.3766 - sparse_categorical_accuracy: 0.8958
Epoch 3/3
100/100 [==============================] - 0s 2ms/step - loss: 0.3185 - sparse_categorical_accuracy: 0.9070


In [29]:
########### using validation_dataset ##########

model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=1, validation_data=val_dataset)


782/782 [==============================] - 2s 3ms/step - loss: 0.3326 - sparse_categorical_accuracy: 0.9060 - val_loss: 0.1942 - val_sparse_categorical_accuracy: 0.9429


In [31]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(
    train_dataset,
    epochs=3,
    # Only run validation using the first 10 batches of the dataset
    # using the `validation_steps` argument
    validation_data=val_dataset,
    validation_steps=10,
)


Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 0.3366 - sparse_categorical_accuracy: 0.9048 - val_loss: 0.3162 - val_sparse_categorical_accuracy: 0.9125
Epoch 2/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1606 - sparse_categorical_accuracy: 0.9528 - val_loss: 0.2346 - val_sparse_categorical_accuracy: 0.9438
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1151 - sparse_categorical_accuracy: 0.9653 - val_loss: 0.1453 - val_sparse_categorical_accuracy: 0.9578


# DATASET
Besides NumPy arrays, eager tensors, and TensorFlow Datasets, it's possible to train a Keras model using Pandas dataframes, or from Python generators that yield batches of data & labels.

In particular, the keras.utils.Sequence class offers a simple interface to build Python data generators that are multiprocessing-aware and can be shuffled.

In general, we recommend that you use:

    1 NumPy input data if your data is small and fits in memory
    2 Dataset objects if you have large datasets and you need to do distributed training
    3 Sequence objects if you have large datasets and you need to do a lot of custom Python-side processing that cannot be done in TensorFlow (e.g. if you rely on external libraries for data loading or preprocessing).


In [35]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence


# Here, `filenames` is list of path to the images
# and `labels` are the associated labels.

class CIFAR10Sequence(Sequence):
    def __init__(self, filenames, labels, batch_size):
        self.filenames, self.labels = filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array([
            resize(imread(filename), (200, 200))
               for filename in batch_x]), np.array(batch_y)

sequence = CIFAR10Sequence(filenames, labels, batch_size)
model.fit(sequence, epochs=10)

NameError: name 'filenames' is not defined

# Callbacks

In [36]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1,
    )
]
model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
    validation_split=0.2,
)

Epoch 1/20
625/625 [==============================] - 2s 2ms/step - loss: 0.3555 - sparse_categorical_accuracy: 0.8989 - val_loss: 0.2447 - val_sparse_categorical_accuracy: 0.9242
Epoch 2/20
625/625 [==============================] - 1s 2ms/step - loss: 0.1670 - sparse_categorical_accuracy: 0.9504 - val_loss: 0.1807 - val_sparse_categorical_accuracy: 0.9438
Epoch 3/20
625/625 [==============================] - 1s 2ms/step - loss: 0.1221 - sparse_categorical_accuracy: 0.9633 - val_loss: 0.1559 - val_sparse_categorical_accuracy: 0.9528
Epoch 4/20
625/625 [==============================] - 1s 2ms/step - loss: 0.0956 - sparse_categorical_accuracy: 0.9709 - val_loss: 0.1415 - val_sparse_categorical_accuracy: 0.9574
Epoch 5/20
625/625 [==============================] - 1s 2ms/step - loss: 0.0788 - sparse_categorical_accuracy: 0.9760 - val_loss: 0.1570 - val_sparse_categorical_accuracy: 0.9540
Epoch 6/20
625/625 [==============================] - 1s 2ms/step - loss: 0.0659 - sparse_categorica

In [37]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []

    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get("loss"))

In [38]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath="mymodel_{epoch}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    )
]
model.fit(
    x_train, y_train, epochs=2, batch_size=64, callbacks=callbacks, validation_split=0.2
)


Epoch 1/2
616/625 [============================>.] - ETA: 0s - loss: 0.3793 - sparse_categorical_accuracy: 0.8916
Epoch 00001: val_loss improved from inf to 0.23974, saving model to mymodel_1
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: mymodel_1\assets
625/625 [==============================] - 3s 5ms/step - loss: 0.3771 - sparse_categorical_accuracy: 0.8923 - val_loss: 0.2397 - val_sparse_categorical_accuracy: 0.9265
Epoch 2/2
622/625 [============================>.] - ETA: 0s - loss: 0.1808 - sparse_categorical_accuracy: 0.9462
Epoch 00002: val_loss improved from 0.23974 to 0.19989, saving model to mymodel_2
INFO:tensorflow:Assets written to: mymodel_2\assets
625/625 [==============================] - 4s 7ms/step - loss: 0.1809 - sparse_categorical_accuracy: 0.94

In [40]:
import os

# Prepare a directory to store all the checkpoints.
checkpoint_dir = "./ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    print(checkpoints)
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()


model = make_or_restore_model()
callbacks = [
    # This callback saves a SavedModel every 100 batches.
    # We include the training loss in the saved model name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + "/ckpt-loss={loss:.2f}", save_freq=100
    )
]
model.fit(x_train, y_train, epochs=1, callbacks=callbacks)


['./ckpt/ckpt-loss=0.31', './ckpt/ckpt-loss=0.32', './ckpt/ckpt-loss=0.33', './ckpt/ckpt-loss=0.34', './ckpt/ckpt-loss=0.35', './ckpt/ckpt-loss=0.37', './ckpt/ckpt-loss=0.38', './ckpt/ckpt-loss=0.40', './ckpt/ckpt-loss=0.42', './ckpt/ckpt-loss=0.45', './ckpt/ckpt-loss=0.48', './ckpt/ckpt-loss=0.53', './ckpt/ckpt-loss=0.60', './ckpt/ckpt-loss=0.72', './ckpt/ckpt-loss=1.02']
Restoring from ./ckpt/ckpt-loss=0.31
1563/1563 [==============================] - 17s 11ms/step - loss: 0.1478 - sparse_categorical_accuracy: 0.9555


In [41]:
initial_learning_rate = 0.1
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)


In [53]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.TensorBoard(
    log_dir="/full_path_to_your_logs",
    histogram_freq=0,  # How often to log histogram visualizations
    embeddings_freq=0,  # How often to log embedding visualizations
    update_freq="epoch",
    )  
]
model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
    validation_split=0.2,
)

Epoch 1/20
625/625 [==============================] - 2s 4ms/step - loss: 0.3693 - sparse_categorical_accuracy: 0.8943 - val_loss: 0.2306 - val_sparse_categorical_accuracy: 0.9282
Epoch 2/20
625/625 [==============================] - 1s 2ms/step - loss: 0.1681 - sparse_categorical_accuracy: 0.9506 - val_loss: 0.1746 - val_sparse_categorical_accuracy: 0.9499
Epoch 3/20
625/625 [==============================] - 1s 2ms/step - loss: 0.1218 - sparse_categorical_accuracy: 0.9636 - val_loss: 0.1593 - val_sparse_categorical_accuracy: 0.9528
Epoch 4/20
625/625 [==============================] - 1s 2ms/step - loss: 0.0968 - sparse_categorical_accuracy: 0.9708 - val_loss: 0.1561 - val_sparse_categorical_accuracy: 0.9543
Epoch 5/20
625/625 [==============================] - 1s 2ms/step - loss: 0.0810 - sparse_categorical_accuracy: 0.9761 - val_loss: 0.1426 - val_sparse_categorical_accuracy: 0.9584
Epoch 6/20
625/625 [==============================] - 1s 2ms/step - loss: 0.0687 - sparse_categorica

In [60]:
%tensorboard --logdir /full_path_to_your_logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 8940.